# DataFrames and broadcasting in Julia

## Dataframes

![](https://dataframes.juliadata.org/stable/assets/logo.png)




- [`DataFrames.jl`](https://github.com/JuliaData/DataFrames.jl) is the primary package for working with tabular data in Julia. 

- It provides functionality similar to that of `pandas` in Python or `data.frames` in R. 

- Lot of correspondence between these three packages!


### Installation

`Dataframes.jl` is not shipped by default with Julia - you need to install it.

```julia-repl
>julia] add DataFrames 
```

In [15]:
cd(@__DIR__)
using Pkg; Pkg.activate(".");
Pkg.instantiate()

  Activating project at `~/Academia/Postdoc_S2z/teaching/iDiv_Julia_workshop/materials/Day1/32_dataframe_tuto`
Precompiling project...
  ✓ Latexify → DataFramesExt
  1 dependency successfully precompiled in 1 seconds. 182 already precompiled. 1 skipped during auto due to previous errors.


### Constructing `DataFrame`s

In [16]:
using DataFrames

df = DataFrame(grp=repeat(1:2, 3), 
                x=6:-1:1, 
                y=4:9, 
                z=[3:7; missing], 
                id='a':'f')

Row,grp,x,y,z,id
,Int64,Int64,Int64,Int64?,Char
1,1,6,4,3,a
2,2,5,5,4,b
3,1,4,6,5,c
4,2,3,7,6,d
5,1,2,8,7,e
6,2,1,9,missing,f


In [17]:
df2 = DataFrame("grp"=>[1, 3], 
                "w"=>[10, 11]) # Pair syntax 

Row,grp,w
,Int64,Int64
1,1,10
2,3,11


In [18]:
names(df)

5-element Vector{String}:
 "grp"
 "x"
 "y"
 "z"
 "id"

Constructing Row by Row

In [19]:
df3 = DataFrame(A=Int[], B=String[])
push!(df3, (1, "M"))

Row,A,B
,Int64,String
1,1,M


### Accessing data

Cell indexing by location

In [20]:
df[2, 2]

5

Row slicing by location

In [21]:
df[2:3, :]

Row,grp,x,y,z,id
,Int64,Int64,Int64,Int64?,Char
1,2,5,5,4,b
2,1,4,6,5,c


Column indexing

In [22]:
df[:, :x]

6-element Vector{Int64}:
 6
 5
 4
 3
 2
 1

In [23]:
df.x

6-element Vector{Int64}:
 6
 5
 4
 3
 2
 1

In [24]:
df[:, [:x, :z]]

Row,x,z
,Int64,Int64?
1,6,3
2,5,4
3,4,5
4,3,6
5,2,7
6,1,missing


Row indexing by label

In [25]:
df[df.id .== 'c', :]

Row,grp,x,y,z,id
,Int64,Int64,Int64,Int64?,Char
1,1,4,6,5,c


Notice the `.` in front of the `==`. More on that in a minute!

### Changing the data stored in a dataframe

In [26]:
df[:,:x] = rand([1,2,3,4],length(df.x))
df

Row,grp,x,y,z,id
,Int64,Int64,Int64,Int64?,Char
1,1,2,4,3,a
2,2,2,5,4,b
3,1,3,6,5,c
4,2,3,7,6,d
5,1,3,8,7,e
6,2,1,9,missing,f


### Common operations

In [28]:
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,grp,1.5,1,1.5,2,0,Int64
2,x,2.16667,1,2.0,4,0,Int64
3,y,6.5,4,6.5,9,0,Int64
4,z,5.0,3,5.0,7,1,"Union{Missing, Int64}"
5,id,,a,,f,0,Char


Reduce multiple values with `skipmissing` (returns memory efficient iterator)


In [29]:
using Statistics
mean(skipmissing(df.z))

5.0

Rename columns

In [30]:
rename(df, :x => :x_new)

Row,grp,x_new,y,z,id
,Int64,Int64,Int64,Int64?,Char
1,1,2,4,3,a
2,2,1,5,4,b
3,1,1,6,5,c
4,2,2,7,6,d
5,1,4,8,7,e
6,2,3,9,missing,f


With the bang operator

In [31]:
rename!(df, :x => :x_new)

Row,grp,x_new,y,z,id
,Int64,Int64,Int64,Int64?,Char
1,1,2,4,3,a
2,2,1,5,4,b
3,1,1,6,5,c
4,2,2,7,6,d
5,1,4,8,7,e
6,2,3,9,missing,f


Drop missing rows

In [32]:
dropmissing(df)

Row,grp,x_new,y,z,id
,Int64,Int64,Int64,Int64,Char
1,1,2,4,3,a
2,2,1,5,4,b
3,1,1,6,5,c
4,2,2,7,6,d
5,1,4,8,7,e


Select unique rows

In [33]:
unique(df)

Row,grp,x_new,y,z,id
,Int64,Int64,Int64,Int64?,Char
1,1,2,4,3,a
2,2,1,5,4,b
3,1,1,6,5,c
4,2,2,7,6,d
5,1,4,8,7,e
6,2,3,9,missing,f


### Grouping data and aggregation
DataFrames.jl provides a groupby function to apply operations over each group independently.

In [34]:
dfg = groupby(df, :grp)

Row,grp,x_new,y,z,id
,Int64,Int64,Int64,Int64?,Char
1,1,2,4,3,a
2,1,1,6,5,c
3,1,4,8,7,e
Row,grp,x_new,y,z,id
,Int64,Int64,Int64,Int64?,Char
1,2,1,5,4,b
2,2,2,7,6,d
3,2,3,9,missing,f


In [35]:
dfg[1]

Row,grp,x_new,y,z,id
,Int64,Int64,Int64,Int64?,Char
1,1,2,4,3,a
2,1,1,6,5,c
3,1,4,8,7,e


Aggregate by groups

In [36]:
combine(groupby(df, :grp), :y => mean)

Row,grp,y_mean
,Int64,Float64
1,1,6.0
2,2,7.0


### Reading a CSV file

To read a parse a CSV and pipe it in a DataFrame, you'll need to install the [`CSV.jl`](https://github.com/JuliaData/CSV.jl) package. CSV.jl is *a fast, flexible delimited file reader/writer for Julia.*

In [48]:
isfile("housing.csv") ||
download("https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv",
    "housing.csv")

"housing.csv"

In [49]:
using CSV
housing_filename = "housing.csv"
CSV.read(housing_filename, DataFrame)

Row,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64,Float64,Float64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.9,5.33,36.2
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222,18.7,394.12,5.21,28.7
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.6,12.43,22.9
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,396.9,19.15,27.1
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93,16.5


Checkout [`CSV.jl` documentation](https://csv.juliadata.org/stable/index.html#Overview) to learn more.

## Broadcasting

In R, something like this works perfectly:
```R
a <- c(0, pi/2, pi, 3*pi/2, 2*pi)
b <- sin(a)
```



But in Julia, the following fails!

In [38]:
a = [0, π/2, π, 3π/2, 2π]
b = sin(a)

MethodError: MethodError: no method matching sin(::Vector{Float64})

Closest candidates are:
  sin(!Matched::Irrational{:π})
   @ Base mathconstants.jl:126
  sin(!Matched::Float16)
   @ Base math.jl:1558
  sin(!Matched::BigFloat)
   @ Base mpfr.jl:801
  ...


- In R, `sin` function works with lists thanks to the fact the there exists a built-in vectorized function `sin`.


Note that the following R user-defined function

```R
# A function that returns the square of a number if it is even, and -1 if odd
even_square <- function(x){
  if(x %% 2 == 0){
    return(x^2)
  } else {
    return(-1)
  }
}
```

would fail with a R list:

```R
# A vector of numbers
nums <- c(1, 2, 3, 4, 5)

even_square(nums) # Error in if (x%%2 == 0) { : the condition has length > 1
```



For the above code to work, you'd need to vectorize the function `even_square` with the `apply` family function - in a similar fashion as you would use the dot operator in Julia

```R
sapply(nums, even_square)
```


- In Julia, you can obtain this behavior by **broadcasting**,
-  allows for **the application of operations to arrays or collections** in a way that is both concise and efficient.


The syntax for broadcasting is the use of the dot notation `.`, followed by the operator.

In [39]:
b = sin.(a)

5-element Vector{Float64}:
  0.0
  1.0
  1.2246467991473532e-16
 -1.0
 -2.4492935982947064e-16

- The dot notation tells Julia to apply the `sin` operator to each element of `a`. 
- This is more natural, because the $sin$ function is mathematically only defined for scalar values.



Similarly, in R you could write something like

```R
b = a + 1
```

but mathematically, this is weird, since adding a scalar to a vector does not make sense! As such, this in Julia fails

In [40]:
b = a + 1

MethodError: MethodError: no method matching +(::Vector{Float64}, ::Int64)
For element-wise addition, use broadcasting with dot syntax: array .+ scalar

Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:587
  +(!Matched::BigInt, ::Union{Int16, Int32, Int64, Int8})
   @ Base gmp.jl:555
  +(!Matched::BigFloat, ::Union{Int16, Int32, Int64, Int8})
   @ Base mpfr.jl:447
  ...


But this works!

In [41]:
b = a .+ 1

5-element Vector{Float64}:
 1.0
 2.5707963267948966
 4.141592653589793
 5.71238898038469
 7.283185307179586

#### Matrix multiplication in R and Julia

In [42]:
a = [1 2; 3 4]
b = [2, 2] 

a * b

2-element Vector{Int64}:
  6
 14

In R this is equivalent to something like

```R
a %*% b
```

while this

In [43]:
a .* b

2×2 Matrix{Int64}:
 2  4
 6  8

Is equivalent in R to 
```R
a * b
```


### Mastering expansion rules with Ref

`r = Ref(x)` 

- create a 0-dimensional container or 0-dimensional reference to `x`
- You can retrieve the `x` with `r[]` (notice that we do not pass any indices in the indexing syntax, as the `r` object is 0-dimensional). 




**Use cases**
- check whether each element of an array is in another array.

In [44]:
mycountries = ["USA", "CHN", "JPN", "DEU", "BRA", "SAU"]
mycountries2 = ["BRA", "FRA", "IRA", "RUS", "CHE"]
x = Bool[]
for c in mycountries2
    if c ∈ mycountries
        push!(x, true)
    else
        push!(x, false)
    end
end
println(x)

Bool[1, 0, 0, 0, 0]


Using broadcasting:

In [45]:
in.(mycountries2, mycountries)

DimensionMismatch: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 5 and 6

That's where `Ref` is useful!

In [46]:
in.(mycountries2, Ref(mycountries))

5-element BitVector:
 1
 0
 0
 0
 0

Equivalent syntax

In [47]:
mycountries2 .∈ Ref(mycountries)

5-element BitVector:
 1
 0
 0
 0
 0

## Your turn!

Try out [some exercises](33_dataframe_exercises.md) with the iris dataset and GBIF 🤩!

Leave out the plotting section, we'll do that after you get a proper introduction to plotting in Julia!

## Acknowledgement and additional resources
- [Bits and pieces of this tutorial have been inspired from the DataFrames.jl documentation](https://dataframes.juliadata.org/stable/)
- [Here is a very detailed tutorial for DataFrames.jl](https://dataframes.juliadata.org/stable/man/basics/#Changing-the-Data-Stored-in-a-Data-Frame)
- [`CSV.jl` documentation](https://csv.juliadata.org/stable/index.html#Overview)